### Part 1 : Word Embeddings and Self attention

In [1]:
!wget http://nlp.stanford.edu/data/wordvecs/glove.6B.zip
!unzip glove.6B.zip

--2021-01-29 17:38:46--  http://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/wordvecs/glove.6B.zip [following]
--2021-01-29 17:38:46--  https://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip [following]
--2021-01-29 17:38:47--  http://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182753 (822M) [applic

In [2]:
import pandas as pd 
glove = pd.read_csv('glove.6B.100d.txt', sep=" ", quoting=3, header=None, index_col=0)
glove.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
the,-0.038194,-0.244870,0.72812,-0.399610,0.083172,0.043953,-0.391410,0.334400,-0.57545,0.087459,0.287870,-0.06731,0.30906,-0.26384,-0.13231,-0.20757,0.33395,-0.338480,-0.31743,-0.48336,0.146400,-0.373040,0.34577,0.052041,0.449460,-0.469710,0.026280,-0.54155,-0.15518,-0.141070,-0.039722,0.28277,0.14393,0.234640,-0.31021,0.086173,0.20397,0.52624,0.171640,-0.082378,...,-0.24978,0.92136,0.034514,0.46745,1.10790,-0.193580,-0.074575,0.233530,-0.052062,-0.220440,0.057162,-0.15806,-0.307980,-0.416250,0.379720,0.150060,-0.532120,-0.205500,-1.25260,0.071624,0.70565,0.497440,-0.42063,0.26148,-1.5380,-0.30223,-0.073438,-0.283120,0.371040,-0.25217,0.016215,-0.017099,-0.389840,0.87424,-0.72569,-0.51058,-0.520280,-0.14590,0.82780,0.270620
",",-0.107670,0.110530,0.59812,-0.543610,0.673960,0.106630,0.038867,0.354810,0.06351,-0.094189,0.157860,-0.81665,0.14172,0.21939,0.58505,-0.52158,0.22783,-0.166420,-0.68228,0.35870,0.425680,0.190210,0.91963,0.575550,0.461850,0.423630,-0.095399,-0.42749,-0.16567,-0.056842,-0.295950,0.26037,-0.26606,-0.070404,-0.27662,0.158210,0.69825,0.43081,0.279520,-0.454370,...,-0.22080,0.18669,0.131770,0.15117,0.71310,-0.352150,0.913480,0.617830,0.709920,0.239550,-0.145710,-0.37859,-0.045959,-0.473680,0.238500,0.205360,-0.189960,0.325070,-1.11120,-0.363410,0.98679,-0.084776,-0.54008,0.11726,-1.0194,-0.24424,0.127710,0.013884,0.080374,-0.35414,0.349510,-0.722600,0.375490,0.44410,-0.99059,0.61214,-0.351110,-0.83155,0.45293,0.082577
.,-0.339790,0.209410,0.46348,-0.647920,-0.383770,0.038034,0.171270,0.159780,0.46619,-0.019169,0.414790,-0.34349,0.26872,0.04464,0.42131,-0.41032,0.15459,0.022239,-0.64653,0.25256,0.043136,-0.194450,0.46516,0.456510,0.685880,0.091295,0.218750,-0.70351,0.16785,-0.350790,-0.126340,0.66384,-0.25820,0.036542,-0.13605,0.402530,0.14289,0.38132,-0.122830,-0.458860,...,-0.55262,0.65000,0.086426,0.39012,1.06320,-0.353790,0.483280,0.346000,0.841740,0.098707,-0.242130,-0.27053,0.045287,-0.401470,0.113950,0.006223,0.036673,0.018518,-1.02130,-0.208060,0.64072,-0.068763,-0.58635,0.33476,-1.1432,-0.11480,-0.250910,-0.459070,-0.096819,-0.17946,-0.063351,-0.674120,-0.068895,0.53604,-0.87773,0.31802,-0.392420,-0.23394,0.47298,-0.028803
of,-0.152900,-0.242790,0.89837,0.169960,0.535160,0.487840,-0.588260,-0.179820,-1.35810,0.425410,0.153770,0.24215,0.13474,0.41193,0.67043,-0.56418,0.42985,-0.012183,-0.11677,0.31781,0.054177,-0.054273,0.35516,-0.302410,0.314340,-0.338460,0.717150,-0.26855,-0.15837,-0.474670,0.051581,-0.33252,0.15003,-0.129900,-0.54617,-0.378430,0.64261,0.82187,-0.080006,0.078479,...,0.04885,0.78267,0.384970,0.42097,0.67882,0.103370,0.632800,-0.026595,0.586470,-0.443320,0.330570,-0.12022,-0.556450,0.073611,0.209150,0.433950,-0.012761,0.089874,-1.79910,0.084808,0.77112,0.631050,-0.90685,0.60326,-1.7515,0.18596,-0.506870,-0.702030,0.665780,-0.81304,0.187120,-0.018488,-0.267570,0.72700,-0.59363,-0.34839,-0.560940,-0.59100,1.00390,0.206640
to,-0.189700,0.050024,0.19084,-0.049184,-0.089737,0.210060,-0.549520,0.098377,-0.20135,0.342410,-0.092677,0.16100,-0.13268,-0.28160,0.18737,-0.42959,0.96039,0.139720,-1.07810,0.40518,0.505390,-0.550640,0.48440,0.380440,-0.002906,-0.349420,-0.099696,-0.78368,1.03630,-0.231400,-0.471210,0.57126,-0.21454,0.359580,-0.48319,1.087500,0.28524,0.12447,-0.039248,-0.076732,...,-0.34780,0.51621,-0.433870,0.36852,0.74573,0.072102,0.279310,0.925690,-0.050336,-0.858560,-0.135800,-0.92551,-0.339910,-1.039400,-0.067203,-0.213790,-0.476900,0.213770,-0.84008,0.052536,0.59298,0.296040,-0.67644,0.13916,-1.5504,-0.20765,0.722200,0.520560,-0.076221,-0.15194,-0.131340,0.058617,-0.318690,-0.61419,-0.62393,-0.41548,-0.038175,-0.39804,0.47647,-0.159830


In [3]:
glove_embedding = {key: val.values for key, val in glove.T.items()}

In [16]:
print(glove_embedding['the'].shape)
glove_embedding['the'][:20]

(100,)


array([-0.038194, -0.24487 ,  0.72812 , -0.39961 ,  0.083172,  0.043953,
       -0.39141 ,  0.3344  , -0.57545 ,  0.087459,  0.28787 , -0.06731 ,
        0.30906 , -0.26384 , -0.13231 , -0.20757 ,  0.33395 , -0.33848 ,
       -0.31743 , -0.48336 ])

In [14]:
import torch

river_emb = torch.tensor(glove_embedding['river'])[None,:]
bank_emb = torch.tensor(glove_embedding['bank'])[None,:]

In [13]:
river_emb.shape

torch.Size([1, 100])

In [15]:
from torch import nn

cosine = nn.CosineSimilarity()
cosine(river_emb,bank_emb)

tensor([0.3345], dtype=torch.float64)

In [42]:
string = ["bank","of","the","river"]
W1 = torch.tensor(glove_embedding[string[0]])
weights = torch.rand((4,100))
reweights_W1 = torch.rand(4)

for i in range(len(string)):
  word = torch.tensor(glove_embedding[string[i]])
  weights[i,:] = word
  reweights_W1[i]=(torch.dot(W1,word))


In [43]:
reweights_W1

tensor([46.4003, 20.8238, 21.5645, 16.5502])

In [44]:
norm_rewieghts = reweights_W1/torch.sum(reweights_W1)

In [45]:
norm_rewieghts

tensor([0.4405, 0.1977, 0.2047, 0.1571])

In [46]:
torch.sum(norm_rewieghts)

tensor(1.)

In [52]:
new_reweighted_W1_vector = norm_rewieghts@weights

In [54]:
cosine(river_emb,new_reweighted_W1_vector[None,:])

tensor([0.5966], dtype=torch.float64)

In [66]:
cosine(torch.tensor(glove_embedding['water'])[None,:],bank_emb)

tensor([0.4943], dtype=torch.float64)

In [69]:
cosine(torch.tensor(glove_embedding['water'])[None,:],new_reweighted_W1_vector[None,:])

tensor([0.5709], dtype=torch.float64)

In [70]:
cosine(torch.tensor(glove_embedding['money'])[None,:],bank_emb)

tensor([0.5718], dtype=torch.float64)

In [71]:
cosine(torch.tensor(glove_embedding['money'])[None,:],new_reweighted_W1_vector[None,:])

tensor([0.6412], dtype=torch.float64)